<a href="https://colab.research.google.com/github/Malebrenchs/Tesis-Valor-Ganado/blob/main/Trazabilidad_QIPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === Colab: Generar diagrama Mermaid desde archivos QIPO/QIPOEC en GitHub ===
# 1) Configura tu repo:
REPO_URL = "https://github.com/Malebrenchs/Tesis-Valor-Ganado.git"  # <-- PON AQUÍ TU URL
BRANCH   = "main"                                              # ej. "main" o "master"

# 2) Opciones:
PROTOCOL_DIR_NAME = "protocol"        # carpeta con tus QIPO/QIPOEC
OUTPUT_FILENAME   = "flowchart.md"    # archivo de salida del diagrama
ENABLE_CLICK_LINKS = True             # True = cada nodo abre su .md en GitHub
# ---------------------------------------------------------------------------

import os, re, subprocess, pathlib, urllib.parse

# Clonar o actualizar repo
repo_dir = pathlib.Path("/content") / pathlib.Path(REPO_URL).stem
if repo_dir.exists():
    print("[INFO] Repo ya existe, haciendo pull...")
    subprocess.run(["git", "-C", str(repo_dir), "fetch"], check=False)
    subprocess.run(["git", "-C", str(repo_dir), "checkout", BRANCH], check=False)
    subprocess.run(["git", "-C", str(repo_dir), "pull"], check=False)
else:
    print("[INFO] Clonando repo...")
    subprocess.run(["git", "clone", "--branch", BRANCH, REPO_URL, str(repo_dir)], check=True)

protocol_dir = repo_dir / PROTOCOL_DIR_NAME
output_file  = repo_dir / OUTPUT_FILENAME

if not protocol_dir.exists():
    raise SystemExit(f"[ERROR] No existe la carpeta '{PROTOCOL_DIR_NAME}' en el repo. Verifica la ruta.")

ID_FROM_FILENAME = re.compile(r'^(QIPOEC-\d+|QIPO-\d+)', re.IGNORECASE)
H1_TITLE         = re.compile(r'^#\s*(QIPOEC-\d+|QIPO-\d+)\s*:\s*(.+)$', re.IGNORECASE)
PREVIOUS_LINE    = re.compile(r'^\s*previous\s*:\s*(QIPOEC-\d+|QIPO-\d+)\s*$', re.IGNORECASE)

def repo_web_url_from_git(url: str) -> str:
    # "https://github.com/user/repo.git" -> "https://github.com/user/repo"
    return url[:-4] if url.endswith(".git") else url

def sanitize_label(s: str) -> str:
    return s.replace('"', "'")

def id_key(i: str) -> int:
    m = re.search(r'-(\d+)$', i)
    return int(m.group(1)) if m else 10**9

def parse_node(md_path: pathlib.Path):
    text = md_path.read_text(encoding="utf-8", errors="ignore")
    node_id, title = None, None

    # 1) Intentar sacar ID y título desde H1 "# QIPO-001: Título"
    for line in text.splitlines():
        m = H1_TITLE.match(line.strip())
        if m:
            node_id = m.group(1).upper()
            title   = m.group(2).strip()
            break

    # 2) Si no hay H1 válido, usar filename
    if node_id is None:
        m = ID_FROM_FILENAME.match(md_path.name)
        if m:
            node_id = m.group(1).upper()
        # Título desde filename (después del primer "_")
        parts = md_path.stem.split("_", 1)
        title = (parts[1] if len(parts) > 1 else md_path.stem).replace("-", " ").strip()

    # 3) Extraer "previous: QIPO-XXX" dentro de la sección "## C"
    prevs, in_c = [], False
    for line in text.splitlines():
        s = line.strip()
        if s.lower().startswith("## c"):
            in_c = True
            continue
        if in_c and s.startswith("## ") and not s.lower().startswith("## c"):
            in_c = False
            continue
        if in_c:
            m = PREVIOUS_LINE.match(s)
            if m:
                prevs.append(m.group(1).upper())

    return node_id, (title or md_path.stem), prevs

# Recorre /protocol y arma nodos/aristas
nodes = {}            # node_id -> {"label": str, "rel_path": str}
edges = set()         # (src, dst)
for md in sorted(protocol_dir.glob("*.md")):
    nid, title, prevs = parse_node(md)
    if not nid:
        continue
    rel = md.relative_to(repo_dir).as_posix()
    nodes[nid] = {
        "label": sanitize_label(f"{nid} {title}"),
        "rel_path": rel
    }
    for p in prevs:
        edges.add((nid, p))
        if p not in nodes:
            nodes[p] = {"label": p, "rel_path": ""}  # placeholder si aún no existe el archivo

# Construir Mermaid
lines = []
lines.append("# Diagrama de trazabilidad QIPO/QIPOEC")
lines.append("")
lines.append("```mermaid")
lines.append("graph TD")

# Declarar todos los nodos (ordenados por número)
for nid, data in sorted(nodes.items(), key=lambda kv: id_key(kv[0])):
    label = data["label"]
    lines.append(f'    {nid}["{label}"]')

# Aristas: viejo -> nuevo
for src, dst in sorted(edges, key=lambda e: (id_key(e[0]), id_key(e[1]))):
    lines.append(f"    {dst} --> {src}")

# Enlaces clicables (si está activado y el repo es público)
if ENABLE_CLICK_LINKS:
    base_web = repo_web_url_from_git(REPO_URL).rstrip("/")
    for nid, data in nodes.items():
        if data["rel_path"]:
            web_url = f"{base_web}/blob/{BRANCH}/{urllib.parse.quote(data['rel_path'])}"
            tooltip = f"Abrir {data['rel_path']}"
            lines.append(f'    click {nid} "{web_url}" "{tooltip}"')

lines.append("```")

# Guardar archivo
output_file.write_text("\n".join(lines), encoding="utf-8")

print(f"[OK] Generado {output_file}")
print("\nVista previa de las primeras líneas:\n")
print("\n".join(lines[:300]))
print("\n— Abre el archivo generado en la columna izquierda (Files) y verás el diagrama en GitHub al subirlo.")


[INFO] Clonando repo...
[OK] Generado /content/Tesis-Valor-Ganado/flowchart.md

Vista previa de las primeras líneas:

# Diagrama de trazabilidad QIPO/QIPOEC

```mermaid
graph TD
    QIPO-001["QIPO-001 Tema General"]
    QIPO-002["QIPO-002 Palabras Clave"]
    QIPO-003["QIPO-003 Repos-Pregrado"]
    QIPO-004["QIPO-004 Repos Maestría"]
    QIPO-005["QIPO-005 Repos Doctorado"]
    QIPO-006["QIPO-006 Procesamiento de Doc's"]
    QIPO-001 --> QIPO-002
    QIPO-002 --> QIPO-003
    QIPO-002 --> QIPO-004
    QIPO-002 --> QIPO-005
    QIPO-003 --> QIPO-006
    QIPO-004 --> QIPO-006
    QIPO-005 --> QIPO-006
    click QIPO-001 "https://github.com/Malebrenchs/Tesis-Valor-Ganado/blob/main/protocol/QIPO-001_TemaInteres.md" "Abrir protocol/QIPO-001_TemaInteres.md"
    click QIPO-002 "https://github.com/Malebrenchs/Tesis-Valor-Ganado/blob/main/protocol/QIPO-002_PalabrasClave.md" "Abrir protocol/QIPO-002_PalabrasClave.md"
    click QIPO-003 "https://github.com/Malebrenchs/Tesis-Valor-Ganado/blob/main